In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("load_date", "")
load_date = dbutils.widgets.get("load_date")

In [0]:
source_df = spark.sql(f"SELECT * FROM PARQUET.`abfss://silver@azurestorageaccnt.dfs.core.windows.net/{load_date}/creditcheck`")

In [0]:
source_df.count()


###Reading Dimension tables

In [0]:
cust_df = spark.read.table("credcheck_catalog.gold.dim_customer")
plan_df = spark.read.table("credcheck_catalog.gold.dim_plan")
device_df = spark.read.table("credcheck_catalog.gold.dim_device")


###Joining dimkeys to the Fact table

In [0]:
joined_df = source_df.join(cust_df, on="customer_id", how="left").join(plan_df, on="plan_code", how="inner").join(device_df, on="device_code", how="left")
display(joined_df)

In [0]:
fact_df = joined_df.select("dimcust_key", "dimplan_key", "dimdevice_key", "creditcheckid", "monthly_fee", "contract_months","approval_amount", "decision_status", "rejection_reason", "last_updated_ts").orderBy("creditcheckid")
display(fact_df)



In [0]:
fact_df.write.format("delta")\
    .mode("append")\
    .option("path", f"abfss://gold@azurestorageaccnt.dfs.core.windows.net/{load_date}/fact_creditcheck")\
    .saveAsTable("credcheck_catalog.gold.fact_creditcheck")